# 📘 NLSQ Quickstart: Your First Curve Fit in 5 Minutes

> Learn the basics of curve fitting with NLSQ through simple, hands-on examples

⏱️ **10-15 minutes** | 📊 **Level: ●○○ Beginner** | 🎓 **No prior curve fitting experience needed**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/01_getting_started/nlsq_quickstart.ipynb)

---

## 🎯 What You'll Learn

By the end of this quickstart, you will:
- ✓ Fit your first curve using NLSQ's `curve_fit` function
- ✓ Understand JAX's JIT compilation and why the first fit is slower
- ✓ Use memory management features for optimal performance
- ✓ See NLSQ's 100x+ speed advantage over SciPy

---

## 💡 Why This Matters

Curve fitting is fundamental to scientific analysis: extracting parameters from experimental data, fitting calibration curves, modeling physical processes. NLSQ makes this **10-300x faster** than traditional methods through GPU acceleration and intelligent compilation.

**Perfect for:**
- 🔬 Scientists analyzing experimental data
- 📊 First-time curve fitting users
- 🚀 SciPy users needing better performance
- 💻 Anyone working with large datasets

---

## 📚 Before You Begin

**First time here?** Perfect! This is exactly where you should start.

**What you need to know:**
- [ ] Python basics (variables, functions)
- [ ] Basic NumPy array operations

**What you don't need to know:**
- ❌ Advanced mathematics or optimization theory
- ❌ GPU programming or CUDA
- ❌ JAX internals or automatic differentiation

**Software requirements:**
- Python >= 3.12
- NLSQ package: `pip install nlsq` ([Installation guide](../../README.md#installation))
- Optional: GPU for maximum performance (works great on CPU too!)

💡 **Tip:** In Google Colab, set Runtime → Change runtime type → GPU for best performance

---

## 1. Installation and Imports

**Important:** NLSQ requires Python 3.12 or higher.

In [ ]:
# Configure matplotlib for inline plotting in VS Code/Jupyter
# MUST come before importing matplotlib
%matplotlib inline

In [ ]:
from IPython.display import display

In [ ]:
# Uncomment to install in Colab/notebook environment
# !pip install nlsq

In [ ]:
# Import NLSQ before JAX (NLSQ configures JAX for 64-bit precision)
import os
import time

import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

from nlsq import CurveFit, __version__

QUICK = os.environ.get("NLSQ_EXAMPLES_QUICK") == "1"


print(f"NLSQ version: {__version__}")

# Import advanced memory management features
from nlsq import (
    MemoryConfig,
    estimate_memory_requirements,
    get_memory_config,
    memory_context,
    set_memory_limits,
)

---

## ⚡ Quick Start (30 seconds)

Let's fit your first curve! We'll fit a linear function to noisy data.

### Step 1: Define your model function

In [ ]:
def linear(x, m, b):
    """Linear function: y = m*x + b"""
    return m * x + b

### Step 2: Create synthetic data

We'll generate data from y = 3x + 5 with some noise.

In [ ]:
# Generate noisy data
length = 1000
x = np.linspace(0, 10, length)
true_params = (3, 5)  # m=3, b=5
y = linear(x, *true_params) + np.random.normal(0, 0.2, size=length)

# Visualize the data
fig = plt.figure(figsize=(10, 5))
plt.plot(x, y, 'o', alpha=0.5, markersize=3, label='Noisy data')
plt.plot(x, linear(x, *true_params), 'r-', linewidth=2, label='True function (y = 3x + 5)')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Noisy Linear Data')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"True parameters: m={true_params[0]}, b={true_params[1]}")

### Step 3: Fit the data!

Now let's use NLSQ to find the best-fit parameters.

In [ ]:
# Create a CurveFit object
jcf = CurveFit()

# Fit the data
popt, pcov = jcf.curve_fit(linear, x, y, p0=(1, 1))  # p0 is initial guess

# Visualize the fit
y_fit = linear(x, *popt)

fig = plt.figure(figsize=(10, 5))
plt.plot(x, y, 'o', alpha=0.5, markersize=3, label='Noisy data')
plt.plot(x, linear(x, *true_params), 'r--', linewidth=2, label='True function')
plt.plot(x, y_fit, 'g-', linewidth=2, label='Fitted function')
plt.xlabel('x')
plt.ylabel('y')
plt.title('NLSQ Curve Fit Results')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n✅ Fitting successful!")
print(f"\nFitted parameters: m={popt[0]:.4f}, b={popt[1]:.4f}")
print(f"True parameters:   m={true_params[0]:.4f}, b={true_params[1]:.4f}")
print(f"\nErrors: Δm={abs(popt[0]-true_params[0]):.4f}, Δb={abs(popt[1]-true_params[1]):.4f}")

✓ **Success!** If your fitted parameters are close to the true values (3, 5), you've completed your first fit!

💡 **What just happened:** NLSQ used the Levenberg-Marquardt algorithm with automatic differentiation to find the parameters (m, b) that minimize the difference between your model and data.

---

## 2. Memory Management and Configuration

NLSQ includes sophisticated memory management features for optimal performance with large datasets.

In [ ]:
# Check current memory configuration
current_config = get_memory_config()
print(f"Current memory limit: {current_config.memory_limit_gb} GB")
print(f"Mixed precision fallback: {current_config.enable_mixed_precision_fallback}")

# Estimate memory requirements for our dataset
n_points = len(x)
n_params = 2  # m and b for linear function
memory_stats = estimate_memory_requirements(n_points, n_params)

print(f"\nMemory estimate for {n_points:,} points, {n_params} parameters:")
print(f"  Total memory needed: {memory_stats.total_memory_estimate_gb:.4f} GB")
print(f"  Recommended chunk size: {memory_stats.recommended_chunk_size:,}")
print(f"  Number of chunks needed: {memory_stats.n_chunks}")

### Temporary Memory Configuration

You can temporarily change memory settings using context managers:

In [ ]:
print("Default memory limit:", get_memory_config().memory_limit_gb, "GB")

# Use a temporary memory configuration
temp_config = MemoryConfig(memory_limit_gb=4.0, enable_mixed_precision_fallback=True)
with memory_context(temp_config):
    print("Inside context memory limit:", get_memory_config().memory_limit_gb, "GB")

print("After context memory limit:", get_memory_config().memory_limit_gb, "GB")

print("\n✓ Context managers allow temporary configuration changes!")

---

## 3. Understanding JAX Compilation

NLSQ uses JAX's JIT (Just-In-Time) compilation for speed. Let's see how this affects fit times.

In [ ]:
def get_random_parameters(mmin=1, mmax=10, bmin=0, bmax=10):
    """Generate random linear parameters"""
    m = mmin + (mmax - mmin) * np.random.random()
    b = bmin + (bmax - bmin) * np.random.random()
    return m, b

# Fit 20 different datasets
length = 300000  # 300K points
x = np.linspace(0, 10, length)

jcf = CurveFit()
nlsq_fit_times = []
nsamples = 21

for i in range(nsamples):
    params = get_random_parameters()
    y = linear(x, *params) + np.random.normal(0, 0.2, size=length)

    start_time = time.time()
    popt, pcov = jcf.curve_fit(linear, x, y, p0=(1, 1))
    nlsq_fit_times.append(time.time() - start_time)

# Visualize
fig = plt.figure(figsize=(12, 5))
plt.plot(nlsq_fit_times, 'o-', linewidth=2, markersize=8)
plt.axhline(np.mean(nlsq_fit_times[1:]), color='r', linestyle='--',
            label=f'Average (after compilation): {np.mean(nlsq_fit_times[1:]):.4f}s')
plt.xlabel('Fit Iteration')
plt.ylabel('Fit Time (seconds)')
plt.title('JAX Compilation Effect: First Fit Slower, Then Fast')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"First fit (with compilation): {nlsq_fit_times[0]:.4f}s")
print(f"Average after compilation:    {np.mean(nlsq_fit_times[1:]):.4f}s")
print(f"\nSpeedup after compilation: {nlsq_fit_times[0] / np.mean(nlsq_fit_times[1:]):.1f}x")

💡 **Key Insight:** The first fit is slow because JAX is compiling (tracing) the functions. After compilation, subsequent fits are **extremely fast** because they reuse the compiled code!

---

## 4. Varying Data Sizes: The Recompilation Problem

What happens if we change the data size for each fit? JAX must recompile for each new array size.

In [ ]:
def get_random_data(length):
    """Generate random linear data"""
    xdata = np.linspace(0, 10, length)
    params = get_random_parameters()
    ydata = linear(xdata, *params) + np.random.normal(0, 0.2, size=length)
    return xdata, ydata

# Test different data sizes
lengths = np.linspace(10**3, 10**6, 20, dtype=int)

jcf = CurveFit()
nlsq_fit_times_varying = []

for length in lengths:
    xdata, ydata = get_random_data(length)
    start_time = time.time()
    popt, pcov = jcf.curve_fit(linear, xdata, ydata, p0=(1, 1))
    nlsq_fit_times_varying.append(time.time() - start_time)

fig = plt.figure(figsize=(12, 5))
plt.plot(lengths, nlsq_fit_times_varying, 'o-', linewidth=2, label='Without fixed size')
plt.xlabel('Data Length')
plt.ylabel('Fit Time (seconds)')
plt.title('Varying Data Size: Recompilation at Each Size')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Total time with recompilation: {np.sum(nlsq_fit_times_varying):.2f}s")
print("\n⚠️ Notice: Every fit is slow because JAX recompiles for each new array size!")

### Solution: Fixed Array Size

NLSQ can use a fixed array size to avoid recompilation!

In [ ]:
# Use fixed array size to avoid recompilation
fixed_length = np.max(lengths)
jcf_fixed = CurveFit(flength=fixed_length)

nlsq_fit_times_fixed = []
for length in lengths:
    xdata, ydata = get_random_data(length)
    start_time = time.time()
    popt, pcov = jcf_fixed.curve_fit(linear, xdata, ydata, p0=(1, 1))
    nlsq_fit_times_fixed.append(time.time() - start_time)

# Compare
fig = plt.figure(figsize=(12, 5))
plt.plot(lengths, nlsq_fit_times_varying, 'o-', linewidth=2, label='Without fixed size (recompiles)')
plt.plot(lengths, nlsq_fit_times_fixed, 's-', linewidth=2, label='With fixed size (no recompile)')
plt.xlabel('Data Length')
plt.ylabel('Fit Time (seconds)')
plt.title('Fixed Array Size Eliminates Recompilation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Total time without fixed size: {np.sum(nlsq_fit_times_varying):.2f}s")
print(f"Total time with fixed size:    {np.sum(nlsq_fit_times_fixed):.2f}s")
print(f"\nSpeedup: {np.sum(nlsq_fit_times_varying) / np.sum(nlsq_fit_times_fixed):.1f}x faster!")

✓ **Success!** By setting `flength=fixed_length`, we avoid recompilation and get consistently fast fits!

💡 **Best Practice:** If you have varying data sizes, use `CurveFit(flength=max_expected_length)`

---

## 5. Fitting Multiple Functions

**Important:** Use separate `CurveFit` objects for different functions to avoid recompilation.

In [ ]:
def quad_exp(x, a, b, c, d):
    """Quadratic + exponential function"""
    return a * x**2 + b * x + c + jnp.exp(d)

# BAD: Using same CurveFit object for different functions
length = 300000
x = np.linspace(0, 10, length)
jcf_shared = CurveFit()

n_runs = 5 if QUICK else 21
linear_params = np.random.random((n_runs, 2))
quad_params = np.random.random((n_runs, 4))

linear_times_bad = []
quad_times_bad = []

for i in range(n_runs):
    y_lin = linear(x, *linear_params[i]) + np.random.normal(0, 0.2, size=length)
    y_quad = quad_exp(x, *quad_params[i]) + np.random.normal(0, 0.2, size=length)

    start = time.time()
    jcf_shared.curve_fit(linear, x, y_lin, p0=(0.5, 0.5))
    linear_times_bad.append(time.time() - start)

    start = time.time()
    jcf_shared.curve_fit(quad_exp, x, y_quad, p0=(0.5, 0.5, 0.5, 0.5))
    quad_times_bad.append(time.time() - start)

fig = plt.figure(figsize=(12, 5))
plt.plot(linear_times_bad, 'o-', label='Linear (shared object)')
plt.plot(quad_times_bad, 's-', label='Quad+Exp (shared object)')
plt.xlabel('Iteration')
plt.ylabel('Fit Time (seconds)')
plt.title('BAD: Sharing CurveFit Object: Constant Recompilation')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Total time (shared object): {np.sum(linear_times_bad + quad_times_bad):.2f}s")
print("\n⚠️ Every fit recompiles because we're alternating between functions!")

In [ ]:
# GOOD: Separate CurveFit objects for each function
jcf_linear = CurveFit()
jcf_quad = CurveFit()

linear_times_good = []
quad_times_good = []

for i in range(n_runs):
    y_lin = linear(x, *linear_params[i]) + np.random.normal(0, 0.2, size=length)
    y_quad = quad_exp(x, *quad_params[i]) + np.random.normal(0, 0.2, size=length)

    start = time.time()
    jcf_linear.curve_fit(linear, x, y_lin, p0=(0.5, 0.5))
    linear_times_good.append(time.time() - start)

    start = time.time()
    jcf_quad.curve_fit(quad_exp, x, y_quad, p0=(0.5, 0.5, 0.5, 0.5))
    quad_times_good.append(time.time() - start)

fig = plt.figure(figsize=(12, 5))
plt.plot(linear_times_good, 'o-', label='Linear (dedicated object)')
plt.plot(quad_times_good, 's-', label='Quad+Exp (dedicated object)')
plt.xlabel('Iteration')
plt.ylabel('Fit Time (seconds)')
plt.title('GOOD: Separate CurveFit Objects: Compile Once, Run Fast')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Total time (shared object):   {np.sum(linear_times_bad + quad_times_bad):.2f}s")
print(f"Total time (separate objects): {np.sum(linear_times_good + quad_times_good):.2f}s")
print(f"\nSpeedup: {np.sum(linear_times_bad + quad_times_bad) / np.sum(linear_times_good + quad_times_good):.1f}x faster!")

💡 **Best Practice:** Create one `CurveFit()` object per unique function you're fitting

---

## 6. NLSQ vs SciPy: Speed Comparison

Let's compare NLSQ's performance against SciPy's `curve_fit`.

In [ ]:
import warnings

from scipy.optimize import OptimizeWarning


def quad_exp_numpy(x, a, b, c, d):
    """NumPy version for SciPy (with overflow protection)"""
    d_clipped = np.clip(d, -700, 700)
    return a * x**2 + b * x + c + np.exp(d_clipped)

length = 300000
x = np.linspace(0, 10, length)

jcf = CurveFit()
all_params = np.random.random((n_runs, 4))

nlsq_times = []
scipy_times = []

# Suppress SciPy covariance warnings (not relevant for performance comparison)
warnings.filterwarnings("ignore", category=OptimizeWarning)

for i in range(n_runs):
    y = quad_exp(x, *all_params[i]) + np.random.normal(0, 0.2, size=length)

    # NLSQ
    start = time.time()
    popt_nlsq, _ = jcf.curve_fit(quad_exp, x, y, p0=(0.5, 0.5, 0.5, 0.5))
    nlsq_times.append(time.time() - start)

    # SciPy
    start = time.time()
    popt_scipy, _ = curve_fit(quad_exp_numpy, x, y, p0=(0.5, 0.5, 0.5, 0.5))
    scipy_times.append(time.time() - start)

fig = plt.figure(figsize=(12, 6))
plt.plot(nlsq_times, 'go-', label='NLSQ', linewidth=2, markersize=8)
plt.plot(scipy_times, 'bs-', label='SciPy', linewidth=2, markersize=8)
plt.axhline(np.mean(nlsq_times[1:]), color='g', linestyle='--', alpha=0.7,
           label=f'NLSQ avg (after compile): {np.mean(nlsq_times[1:]):.4f}s')
plt.axhline(np.mean(scipy_times), color='b', linestyle='--', alpha=0.7,
           label=f'SciPy avg: {np.mean(scipy_times):.4f}s')
plt.xlabel('Fit Iteration')
plt.ylabel('Fit Time (seconds)')
plt.title('NLSQ vs SciPy: 300K Points, 4 Parameters')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Performance Summary:")
print("=" * 60)
print(f"NLSQ first fit (with compilation): {nlsq_times[0]:.4f}s")
print(f"NLSQ average (after compilation):  {np.mean(nlsq_times[1:]):.4f}s")
print(f"SciPy average:                      {np.mean(scipy_times):.4f}s")
print(f"\nSpeedup: {np.mean(scipy_times) / np.mean(nlsq_times[1:]):.1f}x faster than SciPy!")
print("\n💡 By avoiding recompilation and using GPU/JAX, NLSQ achieves massive speedups")

---

## 🎓 What You've Learned

Congratulations! You've just:

1. ✓ **Fit your first curve** with NLSQ in just a few lines of code
2. ✓ **Understood JAX compilation** - first fit compiles, then runs super fast
3. ✓ **Learned memory management** - estimate requirements and configure limits
4. ✓ **Avoided recompilation** - use fixed array sizes and separate CurveFit objects
5. ✓ **Saw 100x+ speedups** vs SciPy on large datasets

**Key takeaways:**
- First fit is slow (compilation), subsequent fits are very fast
- Use `CurveFit(flength=max_size)` for varying data sizes
- Create one `CurveFit()` per unique function
- NLSQ is 10-300x faster than SciPy, especially with GPU

---

## ❓ Common First-Time Questions

**Q: Why was my first fit so slow?**  
A: JAX compiles functions on first use (JIT compilation). Subsequent fits reuse the compiled code and run 100-300x faster! This is normal and expected.

**Q: Do I need a GPU to use NLSQ?**  
A: No! NLSQ works great on CPU. GPU gives extra speedup for large datasets (millions of points), but isn't required.

**Q: How is this different from SciPy's curve_fit?**  
A: Same API and algorithms, but NLSQ uses JAX for:
- Automatic differentiation (no manual Jacobians!)
- JIT compilation for speed
- GPU acceleration
- Result: 10-300x faster on large datasets

**Q: My fitted parameters are slightly different from true values. Is that okay?**  
A: Yes! Small differences are normal with noisy data. If your parameters are within 10-20% of true values with the noise we added, that's excellent!

**Q: Can I use NLSQ with my own custom functions?**  
A: Absolutely! Just define your function using `jax.numpy` instead of `numpy` (or use regular `numpy` for simple functions). NLSQ handles the rest.

**Q: What if I get errors about array sizes?**  
A: Use `CurveFit(flength=max_expected_size)` to set a fixed array size. This avoids recompilation when data sizes vary.

💬 More questions? Check the [FAQ](../../docs/faq.md) or [ask the community](https://github.com/imewei/NLSQ/discussions)

---

## 🗺️ What's Next?

**Ready to learn more?**

**Recommended next steps:**
1. **[Interactive Tutorial](nlsq_interactive_tutorial.ipynb)** (30 min) - Hands-on practice with exercises
2. **[Function Library Demo](../05_feature_demos/function_library_demo.ipynb)** (20 min) - Pre-built models (exponential, Gaussian, etc.)
3. **[Domain Gallery](../04_gallery/README.md)** - See examples from your field

**Got a specific need?**
- **Large dataset (>1M points)?** → [Large Dataset Demo](../02_core_tutorials/large_dataset_demo.ipynb)
- **Want GPU acceleration?** → [Performance Optimization](../02_core_tutorials/performance_optimization_demo.ipynb)
- **Need 2D fitting (images)?** → [2D Gaussian Demo](../02_core_tutorials/nlsq_2d_gaussian_demo.ipynb)
- **Custom algorithms?** → [Custom Algorithms](../03_advanced/custom_algorithms_advanced.ipynb)

**Browse by field:**
- 🧬 [Biology Examples](../04_gallery/biology/) - Dose-response, enzyme kinetics, growth curves
- ⚗️ [Chemistry Examples](../04_gallery/chemistry/) - Reaction kinetics, titrations
- ⚛️ [Physics Examples](../04_gallery/physics/) - Oscillations, decay, spectroscopy
- 🔧 [Engineering Examples](../04_gallery/engineering/) - Calibration, materials, system ID

**Not sure where to go?** Check the [Learning Map](../00_learning_map.ipynb) for guided paths!

---

## 🔗 Additional Resources

**Documentation:**
- [Complete API Documentation](https://nlsq.readthedocs.io/)
- [Installation Guide](../../README.md#installation)
- [Troubleshooting Guide](../03_advanced/troubleshooting_guide.ipynb)
- [FAQ](../../docs/faq.md)
- [Glossary](../../docs/glossary.md)

**Community:**
- 💬 [GitHub Discussions](https://github.com/imewei/NLSQ/discussions) - Ask questions
- 🐛 [GitHub Issues](https://github.com/imewei/NLSQ/issues) - Report bugs
- 📜 [Python Script Version](../../scripts/01_getting_started/nlsq_quickstart.py)

**Research:**
- [JAXFit Paper (arXiv)](https://doi.org/10.48550/arXiv.2208.12187) - Original research
- [Citing NLSQ](../../README.md#citing-nlsq)

---

## 📚 Glossary

**Curve fitting:** Finding the parameters of a mathematical function that best match observed data

**JIT compilation (Just-In-Time):** Converting Python code to optimized machine code at runtime. First run is slower (compilation), subsequent runs are very fast.

**JAX:** Google's library for high-performance numerical computing with automatic differentiation and GPU support

**Automatic differentiation:** Computing derivatives automatically without manual calculation or numerical approximation

**Parameters:** The values in your model function that you're trying to find (e.g., m and b in y = mx + b)

**Initial guess (p0):** Starting values for parameters. NLSQ iteratively improves these to find the best fit.

**Covariance matrix (pcov):** Describes uncertainty in fitted parameters and correlations between them

**GPU:** Graphics Processing Unit - specialized hardware that can perform many calculations in parallel, great for large datasets

[See complete glossary](../../docs/glossary.md)

---

## ✅ Congratulations!

You've completed the NLSQ Quickstart! You now have the foundation to:
- Fit curves to your experimental data
- Optimize NLSQ performance
- Understand JAX compilation benefits
- Choose the right next tutorial for your needs

**Happy fitting!** 🎯